Ejercicio: Entrenar un modelo de regresión lineal múltiple

En este ejercicio, entrenaremos tanto un modelo de regresión lineal simple como un modelo de regresión lineal múltiple, y compararemos su rendimiento utilizando R-Cuadrado.

Carga de datos
Empecemos por echar un vistazo a nuestros datos.

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-illness.csv

#Import the data from the .csv file
dataset = pandas.read_csv('doggy-illness.csv', delimiter="\t")

#Let's have a look at the data
dataset

En este ejercicio intentaremos predecir la temperatura del núcleo a partir de otras características disponibles.

Visualización de datos
Veamos rápidamente qué características parecen tener algún tipo de relación con la temperatura_del_núcleo

In [ ]:
import graphing # Custom graphing code that uses Plotly. See our GitHub repository for details

graphing.box_and_whisker(dataset, "male", "core_temperature", show=True)
graphing.box_and_whisker(dataset, "attended_training", "core_temperature", show=True)
graphing.box_and_whisker(dataset, "ate_at_tonys_steakhouse", "core_temperature", show=True)
graphing.scatter_2D(dataset, "body_fat_percentage", "core_temperature", show=True)
graphing.scatter_2D(dataset, "protein_content_of_last_meal", "core_temperature", show=True)
graphing.scatter_2D(dataset, "age", "core_temperature")

A simple vista, los perros más gordos, viejos y machos parecen tener temperaturas más altas que los perros más delgados, jóvenes o hembras. Los perros que comieron muchas proteínas la noche anterior también parecen encontrarse peor. Las demás características no parecen especialmente útiles.

Regresión lineal simple
Intentemos predecir la temperatura_del_núcleo mediante regresión lineal simple y observemos la R-cuadrado de estas relaciones.

In [ ]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

for feature in ["male", "age", "protein_content_of_last_meal", "body_fat_percentage"]:
    # Perform linear regression. This method takes care of
    # the entire fitting procedure for us.
    formula = "core_temperature ~ " + feature
    simple_model = smf.ols(formula = formula, data = dataset).fit()

    print(feature)
    print("R-squared:", simple_model.rsquared)
    
    # Show a graph of the result
    graphing.scatter_2D(dataset, label_x=feature, 
                                 label_y="core_temperature",
                                 title = feature,
                                 trendline=lambda x: simple_model.params[1] * x + simple_model.params[0],
                                 show=True)

Al desplazarnos por estos gráficos, obtenemos valores R-cuadrado de 0,0002 (porcentaje_grasa_corporal), 0,1 (varón) y 0,26 (edad).

Aunque protein_content_of_last_meal también parece muy prometedora, la relación parece curva, no lineal. Dejaremos esta función por ahora y volveremos a ella en el próximo ejercicio.

R-Cuadrado
Hemos mostrado el valor R-cuadrado de estos modelos y lo hemos utilizado como medida de "corrección" de nuestra regresión, pero ¿qué es?

Intuitivamente, podemos pensar en R-cuadrado como un cociente de cuánto mejor es nuestra línea de regresión que una regresión ingenua que simplemente pasa por la media de todos los ejemplos. A grandes rasgos, el R-cuadrado se calcula tomando la pérdida/error del modelo entrenado y dividiéndola por la pérdida/error del modelo ingenuo. Eso da un rango en el que 0 es mejor y 1 es peor, así que se resta todo de 1 para dar la vuelta a esos resultados.

A continuación, mostramos una vez más el gráfico de dispersión con la edad y la temperatura_del_núcleo, pero esta vez, mostramos dos líneas de regresión. La primera es la línea ingenua que pasa directamente por la media. Esto tiene un R-cuadrado de 0 (ya que no es mejor que sí mismo). Un R-Cuadrado de 1 sería una línea que se ajustara perfectamente a cada ejemplo de entrenamiento. El segundo gráfico muestra nuestra línea de regresión entrenada y una vez más vemos su R-Cuadrado.

In [ ]:
formula = "core_temperature ~ age"
age_trained_model = smf.ols(formula = formula, data = dataset).fit()
age_naive_model = smf.ols(formula = formula, data = dataset).fit()
age_naive_model.params[0] = dataset['core_temperature'].mean()
age_naive_model.params[1] = 0

print("naive R-squared:", age_naive_model.rsquared)
print("trained R-squared:", age_trained_model.rsquared)

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="age", 
                                label_y="core_temperature",
                                title = "Naive model",
                                trendline=lambda x: dataset['core_temperature'].repeat(len(x)), 
                                show=True)
# Show a graph of the result
graphing.scatter_2D(dataset, label_x="age", 
                                label_y="core_temperature",
                                title = "Trained model",
                                trendline=lambda x: age_trained_model.params[1] * x + age_trained_model.params[0])



Regresión lineal múltiple

En lugar de modelarlos por separado, vamos a intentar combinarlos en un único modelo. La grasa corporal no parecía ser útil después de todo, así que vamos a utilizar sólo el sexo masculino y la edad como características.

In [ ]:
model = smf.ols(formula = "core_temperature ~ age + male", data = dataset).fit()

print("R-squared:", model.rsquared)

Al utilizar ambas características al mismo tiempo, obtuvimos un resultado mejor que cualquiera de los modelos de una sola característica (univariante).

¿Cómo podemos ver esto? Pues bien, una regresión lineal simple se dibuja en 2d. Si trabajamos con una variable adicional, añadimos una dimensión y trabajamos en 3D.

In [ ]:
import numpy as np
# Show a graph of the result
# this needs to be 3D, because we now have three variables in play: two features and one label

def predict(age, male):
    '''
    This converts given age and male values into a prediction from the model
    '''
    # to make a prediction with statsmodels, we need to provide a dataframe
    # so create a dataframe with just the age and male variables
    df = pandas.DataFrame(dict(age=[age], male=[male]))
    return model.predict(df)

# Create the surface graph
fig = graphing.surface(
    x_values=np.array([min(dataset.age), max(dataset.age)]),
    y_values=np.array([0, 1]),
    calc_z=predict,
    axis_title_x="Age",
    axis_title_y="Male",
    axis_title_z="Core temperature"
)

# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.age, y=dataset.male, z=dataset.core_temperature, mode='markers')
fig.show()

El gráfico anterior es interactivo. Intente girarlo para ver cómo el modelo (mostrado como un plano sólido) predeciría la temperatura central a partir de diferentes combinaciones de edad y sexo.

Inspeccionar nuestro modelo
Cuando tenemos más de dos características, resulta muy difícil visualizar estos modelos. Normalmente tenemos que mirar los parámetros directamente. Hagámoslo ahora. Statsmodels, una de las bibliotecas habituales de aprendizaje automático y estadística, ofrece un método summary() que proporciona información sobre nuestro modelo.

In [ ]:
# Print summary information
model.summary()

Si nos fijamos en la esquina superior derecha, podemos ver nuestra estadística R-cuadrado que imprimimos anteriormente.

Un poco más abajo y a la izquierda, también podemos ver información sobre los datos con los que hemos entrenado nuestro modelo. Por ejemplo, podemos ver que lo hemos entrenado con 98 observaciones (Nº Observaciones).

Debajo encontramos información sobre nuestros parámetros, en una columna llamada coef (que significa coeficientes, un sinónimo de parámetros en el aprendizaje automático). Aquí podemos ver que el intercepto era de aproximadamente 38 - lo que significa que el modelo predice una temperatura central de 38 para un perro con edad=0 y macho=0. Debajo de esto vemos que el parámetro para la edad es 0,14, lo que significa que por cada año adicional de edad, la temperatura predicha aumentaría 0,14 grados centígrados. Para el macho, podemos ver un parámetro de 0,32, lo que significa que el modelo estima que todos los perros (es decir, donde macho == 1) tienen temperaturas 0,32 grados celsius más altas que las hembras (es decir, donde macho == 0).

Aunque no tenemos espacio aquí para entrar en detalles, la columna P también es muy útil. Nos indica el grado de confianza del modelo en el valor de este parámetro. Como regla general, si el valor p es inferior a 0,05, hay muchas posibilidades de que esta relación sea fiable. Por ejemplo, aquí tanto la edad como el sexo masculino son inferiores a 0,05, por lo que podemos confiar en el uso de este modelo en el mundo real.

Como ejercicio final, hagamos lo mismo con nuestro anterior modelo de regresión lineal simple, relacionando la edad con la temperatura_del_corazón. Lee la siguiente tabla y comprueba lo que puedes deducir de este modelo.

In [ ]:
age_trained_model.summary()

Resumen

Cubrimos los siguientes conceptos en este ejercicio:

Construir modelos de regresión lineal simple y múltiple.
Compare el rendimiento de ambos modelos observando los valores R-Squared.
Modelos inspeccionados para entender cómo funcionan.